In [1]:
!pip install anndata 

import numpy as np 
import torch
import anndata as an

import sys
sys.path.append('../src')

from data import clean_sample
from lightning_train import DataModule
from typing import List

Error processing line 1 of /Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/google_auth-2.6.6-py3.10-nspkg.pth:

  Traceback (most recent call last):
    File "/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


ModuleNotFoundError: No module named 'pandas'

In [2]:
data = an.read_h5ad('../data/retina/retina_T.h5ad')

/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
data.X.shape

(19694, 37475)

In [7]:
currgenes = data.var['gene'].values
refgenes = currgenes[1:]

indices = np.intersect1d(currgenes, refgenes, return_indices=True)[1]

In [8]:
len(set(currgenes).intersection(refgenes))

37474

In [13]:
arr = torch.from_numpy(np.array(
    [[3,2,1,0],
    [1,2,3,4],
    [3,2,1,0],
    [1,2,3,4]]
))

In [50]:
data = an.read_h5ad('../data/dental/human_dental_T.h5ad')

currgenes = data.var.index
refgenes = currgenes[5:]

module = DataModule(
    datafiles=['../data/dental/human_dental_T.h5ad'],
    labelfiles=['../data/dental/labels_human_dental.tsv'],
    class_label='cell_type',
    sep='\t',
    batch_size=16,
    num_workers=0,
)

module.prepare_data()
module.setup()

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


In [42]:
currgenes, refgenes

(Index(['FAM138A', 'OR4F5', 'OR4F29', 'OR4F16', 'FAM87B', 'LINC00115', 'FAM41C',
        'SAMD11', 'NOC2L', 'KLHL17',
        ...
        'AC007325.2', 'BX072566.1', 'AL354822.1', 'AC023491.2', 'AC004556.1',
        'AC233755.2', 'AC233755.1', 'AC240274.1', 'AC213203.1', 'FAM231C'],
       dtype='object', length=21771),
 Index(['LINC00115', 'FAM41C', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'PERM1',
        'HES4', 'ISG15', 'AGRN',
        ...
        'AC007325.2', 'BX072566.1', 'AL354822.1', 'AC023491.2', 'AC004556.1',
        'AC233755.2', 'AC233755.1', 'AC240274.1', 'AC213203.1', 'FAM231C'],
       dtype='object', length=21766))

In [45]:
sample = next(iter(module.trainloader))[0]

sample

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [30]:
def old_clean_sample(
    sample: torch.Tensor,
    refgenes: List[str],
    currgenes: List[str],
) -> torch.Tensor:

    intersection = np.intersect1d(currgenes, refgenes, return_indices=True)
    indices = intersection[1] # List of indices in sorted(currgenes) that equal sorted(refgenes)
    
    axis = (1 if sample.ndim == 2 else 0)
    sample = np.take(sample, indices, axis=axis)
    
    return sample

def new_clean_sample(
    sample: torch.Tensor,
    refgenes: List[str],
    currgenes: List[str],
) -> torch.Tensor:
    indices = np.intersect1d(currgenes, refgenes, return_indices=True)[1]
    if sample.ndim == 2:
        sample = sample[:, indices]
    else:
        sample = sample[indices] # in the case of a 1d array (single row)

    return sample

In [52]:
old = old_clean_sample(data.X, refgenes, currgenes)

In [53]:
new = new_clean_sample(data.X, refgenes, currgenes)

In [54]:
old, new

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [61]:
np.equal(old, new).all()

True

In [62]:
old.shape

(41673, 21766)

In [63]:
len(currgenes)

21771

In [64]:
len(refgenes)

21766